# Overview

This Jupyter Notebook takes in data from a Google Sheet that contains stop change details and their associated high level categories and outputs a JSON file for each line to be used in the MyBus tool.

The output file is used by the MyBus tool's results page and contains the Stop-level changes that are displayed there.

Run all cells to output files in the format `#-changes.json`

As of 5/28/21, this should output data for 141 lines.

In [10]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

DATA_INPUT_PATH = '../data/input/'
DATA_OUTPUT_PATH = '../data/stop-changes/'
SCRATCH_PATH = 'scratch/'
LINES_PATH = '../data/lines.json'
GOOGLE_SHEET_LINK = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQC-r5V2B0xw886UtXO5TU1YXPy_HzyBoQXQOjDWD_qwmBBlvn3fyVsa-BHu0FymPM8gk4O_UxMtJsK/pub?output=csv"

## Load GTFS Data

Use `lines.json`, which was generated from the `line-dropdown` notebook.

Extract `stops.zip` to the scratch folder.

In [11]:
lines = pd.read_json(LINES_PATH)
lines_array = lines.loc[:, 'route_number'].values
lines.head()

,route_id,route_short_name,route_number
0,2,2,2
1,4,4,4
2,10,10,10
3,14,14,14
4,16,16,16


In [12]:
with ZipFile('../data/input/stops.zip', 'r') as zf:
    zf.extractall(SCRATCH_PATH)

STOPS_PATH = SCRATCH_PATH + 'stops.txt'

In [13]:
stops = pd.read_csv(STOPS_PATH,
    usecols=['stop_id','stop_name'],
    dtype={'stop_id':'int','stop_name':'string'})

stops.head()

,stop_id,stop_name
0,1,Paramount / Slauson
1,3,Jefferson / 10th
2,6,120th / Augustus F Hawkins
3,7,120th / Martin Luther King Hospital
4,12,15054 Sherman Way


## Load Stop Changes Data

Data was compiled from spreadsheets and slides provided by Service Planners. Pull the data directly from the Google Sheet.

In [14]:
# stop_changes = pd.read_csv(DATA_INPUT_PATH + 'stop_changes - ALL.csv',
#     usecols={'line', 'stop_id', 'service_canceled', 'service_changed', 'service_replaced', 'stop_canceled', 'stop_relocated', 'route_changed', 'owl_service_canceled'})
stop_changes = pd.read_csv(GOOGLE_SHEET_LINK, usecols={'line', 'direction', 'stop_id', 'service_canceled', 'service_changed', 'service_replaced', 'stop_canceled', 'stop_relocated', 'route_changed', 'owl_service_canceled'})

stop_changes.head()

,line,direction,stop_id,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_canceled
0,2,Westbound,14805,False,False,False,True,False,False,False
1,2,Eastbound,6424,False,False,False,True,False,False,False
2,4,Eastbound,5972,False,False,False,True,False,False,False
3,4,Eastbound,1070,False,False,False,True,False,False,False
4,4,Eastbound,5978,False,False,False,True,False,False,False


In [18]:
# merge with stops.txt to get stop names
#stop_changes.stop_id = stop_changes.stop_id.astype('int')
stop_changes_with_names = pd.merge(stop_changes, stops, how='inner', on='stop_id')
stop_changes_with_names = stop_changes_with_names[['line', 'stop_id', 'stop_name', 'direction', 'service_canceled', 'service_changed', 'service_replaced', 'stop_canceled', 'stop_relocated', 'route_changed', 'owl_service_canceled']]
stop_changes_with_names.to_csv(SCRATCH_PATH + 'stop_changes_with_names.csv')

stop_changes_with_names.head()

,line,stop_id,stop_name,direction,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_canceled
0,2,14805,Sunset / Mapleton,Westbound,False,False,False,True,False,False,False
1,2,6424,Sunset / Mapleton,Eastbound,False,False,False,True,False,False,False
2,4,5972,Santa Monica / 11th,Eastbound,False,False,False,True,False,False,False
3,4,1070,Santa Monica / 17th,Eastbound,False,False,False,True,False,False,False
4,4,5978,Santa Monica / 23rd,Eastbound,False,False,False,True,False,False,False


## Analyze Data

### Stops in `stop_changes` not in GTFS

Outputs results to file `stops-not-in-gtfs.json` in the `scratch` folder.

In [7]:
stops_not_in_gtfs = stop_changes[~stop_changes.stop_id.isin(stops.stop_id)]
stops_not_in_gtfs = stops_not_in_gtfs[['line', 'stop_id']]
stops_not_in_gtfs = stops_not_in_gtfs.sort_values(['line', 'stop_id'])

stops_not_in_gtfs.to_json(SCRATCH_PATH + 'stops-not-in-gtfs.json', orient='records')

stops_not_in_gtfs.head(20)

,line,stop_id
14,4,14381
15,4,14409
32,18,1269
54,152,4580
81,169,12107
88,656,3600
252,690,9031
251,690,10572


### Duplicate `line` & `stop_id` combos

Outputs results to file `stops-changes-duplicates.json` in the `scratch` folder.

In [8]:
# Find duplicate line + stop_id combos

# exclude non-existent stop_ids
stop_changes_existing_stopids = stop_changes.loc[stop_changes['stop_id'] != 0]
stop_changes_existing_stopids = stop_changes_existing_stopids.groupby(['line', 'stop_id', 'direction']).size().reset_index(name="count")
stop_changes_existing_stopids = stop_changes_existing_stopids.loc[stop_changes_existing_stopids['count'] > 1]

# Output file with the stops that have duplicate rows.
filtered_combos = pd.merge(stop_changes_existing_stopids, stop_changes, how='inner', on=['line', 'stop_id', 'direction']).sort_values(by=['line', 'stop_id', 'direction'])
filtered_combos.to_csv(SCRATCH_PATH + 'stop-changes-duplicates.csv')

filtered_combos.head(20)

,count,line,direction,stop_id,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_canceled


## Output Stop Changes Data

Iterate through lines.json and output a file for each line.

In [9]:
# Loop through line numbers and output a file for each line with all data for that line
line_count = 1
stop_count = 0

for line in lines_array:
    subset = stop_changes[stop_changes.line == line]
    filename = DATA_OUTPUT_PATH + str(line) + '-changes.json'
    
    stop_count += len(subset)
    print(subset[['line', 'direction', 'stop_id']])
    subset.to_json(filename, orient='records')
    
    print('********** ' + 'Line ' + filename + ' created (' + str(len(subset)) + ')')
    print('********** ' + str(stop_count)  + ' total stops')
    print('********** ' + str(line_count)  + ' total lines')
    line_count += 1

   line  direction  stop_id
0     2  Westbound    14805
1     2  Eastbound     6424
********** Line ../data/stop-changes/2-changes.json created (2)
********** 2 total stops
********** 1 total lines
    line  direction  stop_id
2      4  Eastbound     5972
3      4  Eastbound     1070
4      4  Eastbound     5978
5      4  Eastbound     5971
6      4  Eastbound     6064
7      4  Eastbound     5983
8      4  Eastbound     5987
9      4  Eastbound     6054
10     4  Eastbound     5984
11     4  Eastbound     6043
12     4  Eastbound     5994
13     4  Eastbound     6046
14     4  Westbound    14381
15     4  Westbound    14409
16     4  Westbound    14427
17     4  Westbound    14374
18     4  Westbound    14425
19     4  Westbound    14365
20     4  Westbound    14434
21     4  Westbound    14366
22     4  Westbound    14364
23     4  Westbound    14444
24     4  Westbound    14351
25     4  Westbound    14358
26     4  Westbound     1091
27     4  Westbound    14352
********** Line ../